### Adjust Word Group Repeat With Youtube Link

In [101]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [102]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [103]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

In [104]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [110]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/Adjust Data Repeat"

Path(path).mkdir(parents=True, exist_ok=True)

#### Repeat Result

In [ ]:
df_word_group_time_loc = pd.read_excel("")  
df_word_group_time_loc

In [ ]:
search_list = df_word_group_time_loc["search_string"].unique().to_list()

In [151]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["search_string","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,search_string,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [155]:
df_result_repeat = pd.DataFrame()
for word_group in search_list:
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   

    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)

df_result_repeat.reset_index(drop=True, inplace=True)
df_result_repeat_join = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"])

df_result_repeat.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time.xlsx", index=False)
df_result_repeat_join.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time_Join.xlsx", index=False)


In [156]:
df_result_repeat

,search_string,start_time,end_time,sentence,video_id,video_url
0,is dat zo,603,605,en op moment dat ik het een keer verkeerd loop...,VjYg2X32ryM,https://www.youtube.com/watch?v=VjYg2X32ryM&t=...
1,is dat zo,95,97,wat is er aan de hand in iran en waarom is dat...,ypikak3H9MI,https://www.youtube.com/watch?v=ypikak3H9MI&t=95s
2,is dat zo,623,624,is dat zo scherp bij transport van mest,USIYNJg2qSw,https://www.youtube.com/watch?v=USIYNJg2qSw&t=...
3,is dat zo,17,19,is dat zo ja ik werd vanochtend wakker om zes ...,NMUAd5R67K0,https://www.youtube.com/watch?v=NMUAd5R67K0&t=17s
4,is dat zo,73,74,is dat zo zijn jullie altijd zo rustig,cUB3dxLXvSc,https://www.youtube.com/watch?v=cUB3dxLXvSc&t=73s
...,...,...,...,...,...,...
112,repeat,5,8,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s
113,weet ik niet,279,281,ja weet ik niet maar ik denk dus dat ik onderd...,GE9qSjs18og,https://www.youtube.com/watch?v=GE9qSjs18og&t=...
114,repeat,5,8,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s
115,weet ik niet,279,281,ja weet ik niet maar ik denk dus dat ik onderd...,GE9qSjs18og,https://www.youtube.com/watch?v=GE9qSjs18og&t=...


In [157]:
(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

4.366666666666666